**Задание 1**

Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

In [1]:
import requests

In [2]:
keywords = ['python', 'парсинг', 'решения']

In [2]:
import time

In [3]:
import pandas as pd

In [5]:
URL = 'https://habr.com/ru/all/'

In [4]:
from bs4 import BeautifulSoup

In [7]:
pages = 3

result_list = []

for i in range(pages + 1):
    if i == 0:
        URL = 'https://habr.com/ru/all/'
    else:
        URL = 'https://habr.com/ru/all/' + 'page' + str(i) +'/' 
    req = requests.get(URL)
    time.sleep(0.3)
    soup = BeautifulSoup(req.text)
    news_blocks = soup.find_all('article', class_='tm-articles-list__item')
    articles_date = list(map(lambda x: x.find('span', class_='tm-article-snippet__datetime-published').find('time').get('title'), news_blocks))
    articles_title = list(map(lambda x: x.find('h2', class_='tm-article-snippet__title').find('span').get_text(), news_blocks))
    articles_text = list(map(lambda x: x.find('div', class_='article-formatted-body').get_text(), news_blocks))
    articles_link = list(map(lambda x: x.find('a', class_='tm-article-snippet__title-link').get('href'), news_blocks))
    result_list = result_list + list(zip(articles_date, articles_title, articles_text, articles_link))

result_list

[('2021-07-12, 19:43',
  'Security Week 28: уязвимости PrintNightmare в деталях',
  'На прошлой неделе специалисты «Лаборатории Касперского» опубликовали разбор двух уязвимостей в системе печати Windows, получившие название PrintNightmare. В среду 7 июля уязвимости закрыли патчем для Windows 7, 10 и серверных ОС начиная с Windows Server 2008 SP2. \n\n\r\nПроблемы в службе печати Windows, приводящие к выполнению произвольного кода, представляют интерес по двум причинам. Во-первых, это очень похоже на более раннюю уязвимость в Windows Print Spooler, использованную в атаке Stuxnet. Во-вторых, первооткрыватели бага случайно выложили PoC-код на GitHub еще в середине июня, когда вендор даже не анонсировал активно эксплуатируемую дыру в Windows.',
  '/ru/company/kaspersky/blog/567376/'),
 ('2021-07-12, 19:34',
  'Внеднение In-app Updates в Android-приложение на React Native',
  'В 2019 году Google выпустила\xa0In-app Updates\xa0— возможность обновлять Android-приложения без перехода в Google 

In [8]:
news = pd.DataFrame()

for e in keywords:
    for el in result_list:
        if e in  el[1].lower() or e in el[2].lower():
            row = {'date': el[0], 'title': el[1], 'link': 'https://habr.com' + el[3]}
            news = pd.concat([news, pd.DataFrame([row])]) 

In [9]:
news

,date,title,link
0,"2021-07-11, 19:31",Модели глубоких нейронных сетей sequence-to-se...,https://habr.com/ru/post/567142/
0,"2021-07-12, 17:49",Путь Namex IXP к IP-фабрикам,https://habr.com/ru/company/otus/blog/567320/
0,"2021-07-12, 15:23",Можно ли сэкономить на СХД?,https://habr.com/ru/company/nubes/blog/567306/
0,"2021-07-12, 17:49",Путь Namex IXP к IP-фабрикам,https://habr.com/ru/company/otus/blog/567320/
0,"2021-07-12, 15:23",Можно ли сэкономить на СХД?,https://habr.com/ru/company/nubes/blog/567306/
0,"2021-07-12, 13:08",Особенности создания синтаксического анализато...,https://habr.com/ru/post/567268/
0,"2021-07-12, 12:26",Оптимизация нагрузки на сеть с помощью разност...,https://habr.com/ru/company/fgts/blog/567098/
0,"2021-07-12, 08:02",Методы укладки Gephi. Force Atlas и Force Atlas 2,https://habr.com/ru/post/567198/


**Задание 2**

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода: EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <дата утечки> - <источник утечки> - <описание утечки>

Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

In [5]:
EMAIL = ['xxx@x.ru', 'yyy@y.com']

In [6]:
url = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'

In [7]:
res = requests.post(url, headers = {'Vaar-Version': '0', 
                                    'Vaar-Header-App-Product-Name': 'hackcheck-web-avast',
                                    'Vaar-Header-App-Build-Version': '1.0.0'
                                   }, 
                    json = {'emailAddresses': EMAIL})
res.text

'{"breaches":{"2":{"breachId":2,"site":"linkedin.com","recordsCount":158591429,"description":"In 2012, online professional networking platform LinkedIn fell victim to a breach of its members\' passwords. Four years later, 117 million email and password combinations from that breach appeared for sale on a dark web marketplace. \\n\\nThe leaked passwords had only been protected by unsalted SHA-1 cryptographic hashing, which prompted LinkedIn to enforce salted hashing after the breach. Russian national Yevgeniy Nikulin was accused of the breach and was extradited from the Czech Republic to the United States as of March 2018.","publishDate":"2016-10-21T00:00:00Z","statistics":{"usernames":0,"passwords":111975337,"emails":158591429}},"3":{"breachId":3,"site":"adobe.com","recordsCount":152046506,"description":"In October of 2013, criminals penetrated Adobe\'s corporate network and the stole source code for several of its software products. The affected products included Adobe\'s ColdFusion w

In [8]:
res.json()

{'breaches': {'2': {'breachId': 2,
   'site': 'linkedin.com',
   'recordsCount': 158591429,
   'description': "In 2012, online professional networking platform LinkedIn fell victim to a breach of its members' passwords. Four years later, 117 million email and password combinations from that breach appeared for sale on a dark web marketplace. \n\nThe leaked passwords had only been protected by unsalted SHA-1 cryptographic hashing, which prompted LinkedIn to enforce salted hashing after the breach. Russian national Yevgeniy Nikulin was accused of the breach and was extradited from the Czech Republic to the United States as of March 2018.",
   'publishDate': '2016-10-21T00:00:00Z',
   'statistics': {'usernames': 0,
    'passwords': 111975337,
    'emails': 158591429}},
  '3': {'breachId': 3,
   'site': 'adobe.com',
   'recordsCount': 152046506,
   'description': "In October of 2013, criminals penetrated Adobe's corporate network and the stole source code for several of its software produc

In [9]:
df = pd.DataFrame(res.json()['breaches'])
df

,2,3,12,15,41,2961,3164,3176,3520,3587,3669,13094,13662,16768,17009,17110,17670,37177
breachId,2,3,12,15,41,2961,3164,3176,3520,3587,3669,13094,13662,16768,17009,17110,17670,37177
site,linkedin.com,adobe.com,vk.com,imesh.com,dropbox.com,cdprojektred.com,cfire.mail.ru,parapa.mail.ru,globalreach.eu,rayli.com.cn,youku.com,myheritage.com,netlog.com,canva.com,zynga.com,azcentral.com,wishbone.io,forums.vkmonline.com
recordsCount,158591429,152046506,110121799,50913222,68591031,1871459,12880232,5029003,921298,4844179,94244880,110041653,52904632,137434010,216159495,705538,37475476,825654
description,"In 2012, online professional networking platfo...","In October of 2013, criminals penetrated Adobe...",Popular Russian social networking platform VKo...,"In June 2016, a cache of over 51 million user ...",Cloud storage company Dropbox suffered a major...,"In March 2016, CDProjektRed.com.com's forum da...","In July and August of 2016, two criminals carr...","In July and August 2016, two criminals execute...","In 2016, Global Reach Technology's database wa...","On an unconfirmed date, Chinese gossip site Ra...",Youku is a large Chinese video content company...,"In October 2017, a customer database belonging...",Netlog (formerly known as Facebox and Bingbox)...,"In May 2019, graphic-design site Canva's datab...","In September 2019, the game developer Zynga wa...","At an unconfirmed date, online Arizona newspap...","In January 2020, the online poll website Wishb...","At an unconfirmed date, the Russian-language m..."
publishDate,2016-10-21T00:00:00Z,2016-10-21T00:00:00Z,2016-10-29T00:00:00Z,2016-10-23T00:00:00Z,2016-10-24T00:00:00Z,2017-01-31T00:00:00Z,2017-02-14T00:00:00Z,2017-02-14T00:00:00Z,2017-03-15T00:00:00Z,2017-03-01T00:00:00Z,2017-03-24T00:00:00Z,2017-11-04T00:00:00Z,2018-02-18T00:00:00Z,2019-06-13T00:00:00Z,2019-10-17T00:00:00Z,2020-01-03T00:00:00Z,2020-05-28T00:00:00Z,2021-02-11T00:00:00Z
statistics,"{'usernames': 0, 'passwords': 111975337, 'emai...","{'usernames': 0, 'passwords': 129430596, 'emai...","{'usernames': 0, 'passwords': 107425364, 'emai...","{'usernames': 50909015, 'passwords': 50913222,...","{'usernames': 0, 'passwords': 68591031, 'email...","{'usernames': 1871410, 'passwords': 1871457, '...","{'usernames': 12880232, 'passwords': 12880222,...","{'usernames': 5029000, 'passwords': 5029003, '...","{'usernames': 13492, 'passwords': 921298, 'ema...","{'usernames': 4841886, 'passwords': 4833694, '...","{'usernames': 0, 'passwords': 94238535, 'email...","{'usernames': 0, 'passwords': 110041647, 'emai...","{'usernames': 0, 'passwords': 52902665, 'email...","{'usernames': 137304104, 'passwords': 61348974...","{'usernames': 214648143, 'passwords': 13476279...","{'usernames': 0, 'passwords': 702971, 'emails'...","{'usernames': 31111127, 'passwords': 10648734,...","{'usernames': 825566, 'passwords': 825654, 'em..."


не смог сообразить как развернуть датафрейм